# Week 3 — Dynamics (Expanded)

**Learning objectives:**
- Detailed derivations and interactive simulations.



In [ ]:
# Install in Colab if needed:
# !pip install sympy numpy scipy matplotlib ipywidgets
# --- Auto GitHub token detection (Colab only) ---
try:
    from google.colab import userdata
    import os
    if 'GITHUB_TOKEN' in userdata:
        token = userdata.get('GITHUB_TOKEN')
        os.environ['GITHUB_TOKEN'] = token
        print('✅ Loaded GITHUB_TOKEN from Colab secrets.')
    else:
        print('⚠️ GITHUB_TOKEN not found in Colab secrets. Use userdata.set_secret("GITHUB_TOKEN").')
except Exception as e:
    print('Not running in Colab or userdata unavailable:', e)

# git config sample (uncomment to use)
# !git config --global user.name 'YourName'
# !git config --global user.email 'you@example.com'

## Lagrangian derivation and computed-torque simulation (2-DOF)

In [ ]:
import sympy as sp
q1,q2,dq1,dq2,l1,l2,m1,m2,I1,I2,g = sp.symbols('q1 q2 dq1 dq2 l1 l2 m1 m2 I1 I2 g', real=True)
q = sp.Matrix([q1,q2]); dq = sp.Matrix([dq1,dq2])
x1 = (l1/2)*sp.cos(q1); y1 = (l1/2)*sp.sin(q1)
x2 = l1*sp.cos(q1) + (l2/2)*sp.cos(q1+q2); y2 = l1*sp.sin(q1) + (l2/2)*sp.sin(q1+q2)
J1 = sp.Matrix([x1,y1]).jacobian(q); J2 = sp.Matrix([x2,y2]).jacobian(q)
v1 = J1*dq; v2 = J2*dq
omega1 = dq1; omega2 = dq1 + dq2
T = sp.simplify(sp.Rational(1,2)*m1*(v1.dot(v1)) + sp.Rational(1,2)*I1*omega1**2 + sp.Rational(1,2)*m2*(v2.dot(v2)) + sp.Rational(1,2)*I2*omega2**2)
V = sp.simplify(m1*g*y1 + m2*g*y2)
M = sp.zeros(2,2)
dqs = [dq1,dq2]; qs=[q1,q2]
for ii in range(2):
    for jj in range(2):
        M[ii,jj] = sp.simplify(sp.diff(sp.diff(T, dqs[ii]), dqs[jj]))
display(sp.simplify(M))

In [ ]:
# Numeric lambdify + quick PD simulation example (Colab)
M_func = sp.lambdify((q1,q2,l1,l2,m1,m2,I1,I2), M, 'numpy')
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
params = dict(l1=1.0,l2=0.8,m1=1.0,m2=0.8,I1=0.01,I2=0.01,g=9.81)
def M_num(qv):
    return np.array(M_func(qv[0], qv[1], params['l1'], params['l2'], params['m1'], params['m2'], params['I1'], params['I2']), dtype=float)
print('Call M_num([q1,q2]) to get numeric inertia matrix')